In [3]:
import cv2
from ultralytics import YOLO
import soundfile as sf
import sounddevice as sd
import threading
import time
import mediapipe as mp

In [ ]:
look_straight_audio_filepath = "../../assets/audios/look_straight.wav"
not_use_phone_audio_filepath = "../../assets/audios/not_use_phone.wav"

# Global variables to manage sound playback
is_playing = False
last_played_time = 0
debounce_time = 1 # seconds

yolo_model = YOLO("yolo11s.pt")
# yolo_model = YOLO("yolov8n.pt")

# Khởi tạo Mediapipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

In [5]:
def sound_audio(audio_file_path):
    global is_playing
    data, samplerate = sf.read(audio_file_path)
    is_playing = True
    sd.play(data, samplerate)
    sd.wait()
    is_playing = False

# Hàm để phát âm thanh trong luồng riêng
def play_sound_async(audio_file_path):
    global is_playing, last_played_time
    current_time = time.time()
    
    # Chỉ phát âm thanh nếu không có âm thanh nào đang phát và đủ thời gian chờ
    if not is_playing and (current_time - last_played_time > debounce_time):
        last_played_time = current_time
        sound_thread = threading.Thread(target=sound_audio, args=(audio_file_path,))
        sound_thread.start()

### Apply frame skip to optimize performance

In [6]:
import os
from datetime import datetime

In [ ]:
cap = cv2.VideoCapture(0)  # Mở camera
if not cap.isOpened():
    print("Không thể mở video hoặc camera.")
    exit()
    
frame_count = 0
frame_skip = 3  # Chỉ dự đoán mỗi 2 khung hình

# Biến để lưu thông tin bounding box trước đó
last_boxes = []

# Biến để theo dõi thời gian đã trôi qua
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Hiển thị ngay lập tức frame dù chưa bắt đầu quá trình detect
    cv2.imshow("YOLOv8 Cell Phone Detection", frame)

    # Kiểm tra nếu đã qua 3 giây, bắt đầu dự đoán
    if time.time() - start_time >= 3:
        frame_count += 1
        if frame_count % frame_skip == 0:
            # Chạy dự đoán chỉ mỗi 3 khung hình
            results = yolo_model.predict(frame, conf=0.6, classes=[67], verbose=False)

            # Cập nhật lại bounding box từ lần dự đoán mới
            last_boxes = []
            for box in results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                conf = float(box.conf[0])
                cls_id = int(box.cls[0])
                last_boxes.append((x1, y1, x2, y2, conf))

            if len(results[0].boxes) > 0:
                play_sound_async(not_use_phone_audio_filepath)
                continue

            # Check khuôn mặt bằng Mediapipe
            face_results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if (not face_results.detections) or face_results.detections[0].score[0] < 0.5:
                print("Không phát hiện khuôn mặt.")
                play_sound_async(look_straight_audio_filepath)
                continue
            else:
                score = face_results.detections[0].score[0]
                print("Confidence:", score)


    # Nếu không phải khung hình đang dự đoán, vẽ bounding box từ lần trước
    if last_boxes:
        for (x1, y1, x2, y2, conf) in last_boxes:
            # Vẽ bounding box và nhãn
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"Cell phone {conf:.2f}",
                (int(x1), int(y1) - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )

    # Hiển thị khung hình với bounding box
    cv2.imshow("YOLOv8 Cell Phone Detection", frame)

    # Điều kiện thoát khi nhấn 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()